# 6. Osztályok, filekezelés
_2021.10.14_

Tartalom:
- Osztályok
- Package/module készítése
- Fájlkezelés (alapértelmezett)
- Hibakezelés
- Datetime

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 6.1 Osztályok, öröklődés
Nagyobb alkalmazások fejlesztése során célszerű az egybe tartozó részeket osztályokba szervezni.
### 6.1.1 Scope, namespaces
A programunk logikailag egymáshoz tartozó részét egy __namespace__ (névtér alá) célszerű elhelyezni. A __scope__ (magyarul láthatóság) a változók és függvények láthatóságára utal. A függvényen vagy osztályon kívül definiált változóknak és függvényeknek globális a láthatóságuk. A __global__ kulcsszóval tudjuk elérni a globális változókat függvényen belülről. A __nonlocal__ kulcsszóval el tudjuk értni a nem globális és nem lokális változókat.

In [2]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


### 6.1.2 Classes
Az osztályok fogják össze a funkcionálisan egybe tartozó változókat és függvényeket. Általánosságban az osztályok példányosíthatók és más osztályok származtathatók belőlük. Az osztályhoz tartozó függvények első paramétere _self_ kell legyen, ugyanis ezen keresztül érhető el a példányosított osztály változói és függvényei.

In [3]:
class MyClass:
    my_var = 12345

    def my_func(self):
        return 'test'

In [10]:
# a MyClass osztály példányosítása
# az x változó a MyClass egy példányára mutat
x = MyClass()

__init__ function called


In [5]:
# osztályváltozó elérése
x.my_var

12345

In [6]:
x.my_func()

'test'

### 6.1.3 Instatiation
A példányosítás során az osztályról készítünk egy másolatot, amelyet eltérő adatokkal tölthetünk fel.

In [7]:
class MyClass:
    my_var = 12345
    def __init__(self):
        print('__init__ function called')
        self.my_var = 54321

    def my_func(self):
        return 'test'

In [8]:
x = MyClass()

__init__ function called


In [9]:
x.my_var

54321

### 6.1.4 Class & instance variables

In [ ]:
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

In [ ]:
d = Kutya('Buksi')
e = Kutya('Blöki')

In [ ]:
d.fajta, d.nev

In [ ]:
e.fajta, e.nev

> **1. feladat**: Egészítsd ki a Kutya osztályt egy tanul_trukk függvénnyel, amely a paraméterként átadott trükköt hozzáadja az adott kutya megtanult trükkjei közé!

Rossz megoldás

In [ ]:
class Kutya:
    fajta = 'kutya' # osztályváltozó
    trukkok = []

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [ ]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [ ]:
d.trukkok

In [ ]:
e.trukkok

Jó megoldás

In [ ]:
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó
        self.trukkok = []

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [ ]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [ ]:
d.trukkok

In [ ]:
e.trukkok

### 6.1.5 Inheritance

In [ ]:
class Jarmu:
    def __init__(self, szin):
        self.szin = szin
    
    def megjelenit(self):
        print('A jármű színe: {:}'.format(self.szin))

In [ ]:
j = Jarmu('piros')
j.megjelenit()

In [ ]:
class Auto(Jarmu):
    def __init__(self, szin, kerekek_szama):
        Jarmu.__init__(self, szin)
        #super().__init__(szin)
        
        self.kerekek_szama = kerekek_szama

    def megjelenit(self):
        print('A jármű színe: {:}\nKerekeinek száma: {:}'.format(self.szin, self.kerekek_szama))

class Repulo(Jarmu):
    pass

In [ ]:
a = Auto('kek', 4)

In [ ]:
a.megjelenit()

> **2. feladat**: Fejezd be a Repulo osztályt, paraméterként adj hozzá egy Boolean típusú változót, amely azt tárolja, hogy a repülőgép futóműve behúzható-e!

In [ ]:
class Repulo(Jarmu):
    def __init__(self, szin, behuzhato):
        super().__init__(szin)
        
        self.behuzhato = behuzhato

    def megjelenit(self):
        print('A jármű színe: {:}\nFutómű behúzható: {:}'.format(self.szin, self.behuzhato))

In [ ]:
r = Repulo('zöld', False)
r.megjelenit()

## 6.2 Filekezelés
Olvassuk be a 6.2_airtravel.csv file tartalmát, amely a havi transzatlanti polgári repülések számát tartalmazza. Az egyes számok ezer utast jelentenek. A file-ok megnyitására az **open(filenév, mód)** parancs használható, ahol a
- _filenév_: a megnyitni kívánt file neve
- _mód_:
    - r: reading, megnyitás olvasásra (alapértelmezett)
    - w: writing: megnyitás írásra
    - a: append: megnyitás hozzáfűzésre
    - r+: megnyitás olvasásra és írásra

In [ ]:
file = open('6.2_airtravel.csv', 'r')

In [ ]:
print (file.read())

In [ ]:
for each in file:
    print (each)

In [ ]:
file = open('test.txt', 'w', encoding='utf-8')
file.write("Szöveg írása fileba")
file.close()

In [ ]:
file = open('test.txt', 'a', encoding='utf-8')
file.write("\nMég egy sor")
file.close()

A with parancs használatával nem szükséges lezárni a file-t, mivel az automatikusan lezáródik.

In [ ]:
with open("test.txt", encoding = 'utf-8') as file:  
    data = file.read()

In [ ]:
data

In [ ]:
f = open("test.txt", encoding = 'utf-8')

In [ ]:
f.read(6)

In [ ]:
f.read(7)

In [ ]:
f.tell()

In [ ]:
f.read()

In [ ]:
f.tell()

In [ ]:
f.seek(0)

In [ ]:
f.read(13)

In [ ]:
f.seek(0)

In [ ]:
f.readline()

In [ ]:
f.readline()

In [ ]:
f.seek(0)

In [ ]:
f.readlines()

## 6.3 Hibakezelés

In [ ]:
10/0

In [ ]:
try:
    print(10/0)
except ZeroDivisionError:
    print("Véletlenül 0-val osztottunk")

In [ ]:
raise NameError('Hello')

In [ ]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")

In [ ]:
divide(2, 1)

In [ ]:
divide(2, 0)

In [ ]:
divide("2", "1")

> **1. feladat**: Egészítsd ki a divide függvényt, hogy helytelen típusú input esetén is dobjon kivételt!

In [ ]:
def divide(x, y):
    try:
        result = x / y
    except TypeError:
        print("Az input paraméterek típusa vagy float vagy integer kell legyen.")
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")

## 6.4 Dátumkezelés

### 6.4.1 Date

In [ ]:
from datetime import date

In [ ]:
datetime.date.today()

In [ ]:
a = date(2019, 4, 13)
print(a)

### 6.4.2 Time

In [ ]:
from datetime import time

In [ ]:
a = datetime.datetime.now().time()
print(a)

In [ ]:
a.hour, a.minute, a.second, a.microsecond

### 6.4.3 Datetime

In [ ]:
import datetime

x = datetime.datetime.now()

In [ ]:
x

In [ ]:
x.year

In [ ]:
x.strftime("%A")

In [ ]:
x.strftime("%Y-%m-%d %H:%M:%S")

### 6.4.4 Timestamp
[epochconverter.com](epochconverter)

In [ ]:
timestamp = 1634142950
datetime.datetime.fromtimestamp(timestamp)

### 6.4.5 Timedelta

In [ ]:
from datetime import datetime, date, timedelta

In [ ]:
t1 = date(year = 2021, month = 7, day = 12)
t2 = date(year = 2020, month = 12, day = 23)
t3 = t1 - t2

In [ ]:
t3, t3.days

In [ ]:
t1 = datetime.now()
t2 = timedelta(days = 14)
t3 = t1 - t2

In [ ]:
t3

In [ ]:
t2.total_seconds()

## 6.5 Package
[Python Modules and Packages – An Introduction](https://realpython.com/python-modules-packages/)

Modular programming refers to the process of breaking a large, unwieldy programming task into separate, smaller, more manageable subtasks or modules. Individual modules can then be cobbled together like building blocks to create a larger application.

There are several advantages to modularizing code in a large application:

Simplicity: Rather than focusing on the entire problem at hand, a module typically focuses on one relatively small portion of the problem. If you’re working on a single module, you’ll have a smaller problem domain to wrap your head around. This makes development easier and less error-prone.

Maintainability: Modules are typically designed so that they enforce logical boundaries between different problem domains. If modules are written in a way that minimizes interdependency, there is decreased likelihood that modifications to a single module will have an impact on other parts of the program. (You may even be able to make changes to a module without having any knowledge of the application outside that module.) This makes it more viable for a team of many programmers to work collaboratively on a large application.

Reusability: Functionality defined in a single module can be easily reused (through an appropriately defined interface) by other parts of the application. This eliminates the need to duplicate code.

Scoping: Modules typically define a separate namespace, which helps avoid collisions between identifiers in different areas of a program. (One of the tenets in the Zen of Python is Namespaces are one honking great idea—let’s do more of those!)

There are actually three different ways to define a module in Python:

A module can be written in Python itself.
A module can be written in C and loaded dynamically at run-time, like the re (regular expression) module.
A built-in module is intrinsically contained in the interpreter, like the itertools module.